<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Random-Forest-regression-and-SHAP-for-feature-importance-measurement" data-toc-modified-id="Random-Forest-regression-and-SHAP-for-feature-importance-measurement-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Random Forest regression and SHAP for feature importance measurement</a></span></li></ul></div>

## Random Forest regression and SHAP for feature importance measurement

In [ ]:


def process_rf_shap(indata, out_path):
    """
    indata:      input shapefile dataset         --string
    out_path:    directory for output files      --string
    """
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import mean_squared_error, r2_score
    from sklearn.model_selection import RandomizedSearchCV, train_test_split
    from sklearn.preprocessing import StandardScaler
    from matplotlib import pyplot as plt
    import geopandas as gpd
    import os
    import shap

    # Load dataset
    df = gpd.read_file(indata)

    # Loop through cluster values
    for id in set(df.cluster.values):
        
        # Extract rows that belong to the current cluster
        df_clst = df[df["cluster"] == id]
        
        # Define selected features
        col_selec = ['kNDVI',
                     'Tmax', 'Tmin', 'Prcp',
                     'Forest', 'Shrub', 'Crop', 'Wetland',
                     'Shannon',
                     'Elevation', 'Aspect', 'Slope', 'TWI',
                     'Clay_mean', 'Sand_mean', 'BD_mean', 'PH_mean', 'SOC_mean', 'TN_mean',
                     'Inundation',
                     'Distance'
                    ] 
        
        # Define training and target data
        X = df_clst[col_selec]
        y = df_clst.Soil_Rs

        # Split the dataset into training and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

        print('Shape of train/test:', X_train.shape, X_test.shape)

        # Define the parameter distributions for RandomizedSearchCV
        distributions = dict(n_estimators=[100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200],
                             max_depth=[3, 6, 9, 12, 15, 18, 21],
                             min_samples_split=[2, 4, 6, 8, 10, 12, 14, 18, 20, 22, 
                                                24, 26, 28, 30, 32, 34, 36, 38, 40],
                             min_samples_leaf=[1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21],
                             max_features=['log2', 'sqrt', None])

        # Instantiate a random forest regressor
        regr = RandomForestRegressor(random_state=42)

        # Define scoring metrics
        score_ = ['r2', 'neg_root_mean_squared_error']

        # Perform randomized search for hyperparameter tuning
        optimal_model = RandomizedSearchCV(estimator=regr,
                                           param_distributions=distributions,
                                           n_iter=2,
                                           scoring=score_,
                                           n_jobs=-1,
                                           refit='neg_root_mean_squared_error',
                                           cv=10,
                                           verbose=1,
                                           return_train_score=True)

        # Fit the optimal model
        optimal_model.fit(X_train, y_train)

        # Create a RandomForestRegressor model using the best_params_ from RandomizedSearchCV
        model = RandomForestRegressor(**optimal_model.best_params_)

        # Train the RandomForestRegressor model
        model.fit(X_train, y_train)

        # Interpret the model using SHAP feature importance
        explainer = shap.TreeExplainer(model)

        # Calculate SHAP values using the full data
        shap_values = explainer(X)

        # Plot the SHAP summary plot
        shap.summary_plot(shap_values,
                          features=X,
                          plot_type='bar',
                          max_display=X.shape[1],
                          show=False)

        # Configure the plot
        fig, ax = plt.gcf(), plt.gca()
        fig.set_size_inches(1.5, 2.2)
        ax.tick_params(axis='both', which='major', labelsize=6.5)
        ax.set_xlabel("Mean of |SHAP value|", fontsize=8)
        ax.set_ylabel("Feature importance", fontsize=8)
        plt.grid(linestyle='-.', axis='y', linewidth=0.1)

        ax.annotate(f"cluster{id}", xy=(0.35, 0.05), xycoords='axes fraction', fontsize=6.5)

        # Save the plot
        plt.savefig(os.path.join(out_path, f'shap_bar_{id}.png'),
                    format='png',
                    dpi=200,
                    bbox_inches='tight')

        # Display the plot
        plt.show()

        # Define the list of lines to write out
        lines = ['---RandomizedSearchCV---',
                 f'mean-CV-score-of-best_estimator: {optimal_model.best_score_}',
                 f'best_estimator_: {optimal_model.best_estimator_}',
                 f'best_params_: {optimal_model.best_params_}',
                 "\n",
                 '---RandomForestRegressor---',
                 f"train-model.score: {model.score(X_train, y_train).round(4)}  #r2 of regression",
                 f"test-model.score: {model.score(X_test, y_test).round(4)}  #r2 of regression",
                 f"train-r2:  {r2_score(y_train, model.predict(X_train))}",
                 f"test-r2:  {r2_score(y_test, model.predict(X_test))}",
                 f"train-MSE: {mean_squared_error(model.predict(X_train), y_train, squared=True)}",
                 f"train-RMSE: {mean_squared_error(model.predict(X_train), y_train, squared=False)}",
                 f"test-MSE: {mean_squared_error(model.predict(X_test), y_test, squared=True)}",
                 f"test-RMSE: {mean_squared_error(model.predict(X_test), y_test, squared=False)}"]

        # Write out the results to a text file
        out_name = os.path.join(out_path, f'RF_best_{id}.txt')
        print(out_name)
        if os.path.exists(out_name):
            os.remove(out_name)
        if not os.path.exists(out_name):
            with open(out_name, 'a') as txt:
                for line in lines:
                    txt.write(line)
                    txt.write('\n')

        print(f"===cluster_{id} done===!")
        
# Execute the function
indata = "/home/compass/CPB_vector.shp"  
out_path = '/home/compass/out'
process_rf_shap(indata, out_path)